In [62]:
#pip install google-cloud-vision
#pip install Pillow
#pip install piexif

In [63]:
from google.cloud import vision

In [64]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:/Users/Patryk/Desktop/deft-station-375717-6847447a0df3.json'

In [65]:
def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations
    #print(labels)
    print('Labels:')

    for label in labels:
        print(label.description + " "+ str(label.score))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
                
def detect_labels(path):
    """Detects labels in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [66]:
detect_labels_uri('https://www.ladysmithchronicle.com/wp-content/uploads/2019/07/17648894_web1_190711-LCH-Garbage-Forests1-640x427.jpg')
#detect_labels(path)

Labels:
Plant 0.8873489499092102
Pollution 0.8469559550285339
Grass 0.821639895439148
Groundcover 0.789735734462738
Gas 0.7763906717300415
Waste 0.7491664886474609
Plastic 0.7215628027915955
Litter 0.7000278234481812
Soil 0.6741319298744202
Event 0.6322991251945496


In [67]:
def localize_objects_uri(uri):
    """Localize objects in the image on Google Cloud Storage

    Args:
    uri: The path to the file in Google Cloud Storage (gs://...)
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = uri

    objects = client.object_localization(
        image=image).localized_object_annotations

    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))
            
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))

In [68]:
localize_objects_uri('https://www.ladysmithchronicle.com/wp-content/uploads/2019/07/17648894_web1_190711-LCH-Garbage-Forests1-640x427.jpg')
#localize_objects(path)

Number of objects found: 0


In [69]:
from PIL import Image
from PIL.ExifTags import TAGS

In [70]:
imagename = "C:/Users/Patryk/Desktop/17648894_web1_190711-LCH-Garbage-Forests1.jpg"
#https://drive.google.com/file/d/1i0hYkcI7nN4fPDAuEhJolDFU7mfm7kBB/view?usp=share_link zdj z lokalizacją gps w metadanych

# read the image data using PIL
image = Image.open(imagename)

In [71]:
import piexif

codec = 'ISO-8859-1'

def exif_to_tag(exif_dict):
    exif_tag_dict = {}
    thumbnail = exif_dict.pop('thumbnail')
    exif_tag_dict['thumbnail'] = thumbnail.decode(codec)

    for ifd in exif_dict:
        exif_tag_dict[ifd] = {}
        for tag in exif_dict[ifd]:
            try:
                element = exif_dict[ifd][tag].decode(codec)

            except AttributeError:
                element = exif_dict[ifd][tag]

            exif_tag_dict[ifd][piexif.TAGS[ifd][tag]["name"]] = element

    return exif_tag_dict

In [74]:
exif_dict = piexif.load(image.info.get('exif'))
exif_dict = exif_to_tag(exif_dict)
print(exif_dict['GPS'])

{'GPSLatitudeRef': 'N', 'GPSLatitude': ((52, 1), (190348191, 10000000), (0, 1)), 'GPSLongitudeRef': 'E', 'GPSLongitude': ((20, 1), (416055962, 10000000), (0, 1))}


In [75]:
#Wysokosc i szerokosc geograficzna pobrana z metadanych zdj
print(str(exif_dict['GPS'].get('GPSLatitude')[0][0])+" "+str(exif_dict['GPS'].get('GPSLatitude')[1][0]))
print(str(exif_dict['GPS'].get('GPSLongitude')[0][0])+" "+str(exif_dict['GPS'].get('GPSLongitude')[1][0]))

52 190348191
20 416055962
